##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 가중치 클러스터링 종합 가이드

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

여기서는 TensorFlow 모델 최적화 도구 키트의 일부인 *가중치 클러스터링*에 대한 종합 가이드를 제공합니다.

이 페이지는 다양한 사용 사례를 문서화하고 각각에 대해 API를 사용하는 방법을 보여줍니다. 필요한 API를 알고 나면, [API 문서](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/clustering)에서 매개변수와 하위 수준의 세부 정보를 찾아보세요.

- 가중치 클러스터링의 이점과 지원되는 기능을 보려면 [개요](https://www.tensorflow.org/model_optimization/guide/clustering)를 참조하세요.
- 단일 엔드 투 엔드 예는 [가중치 클러스터링 예](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_example)를 참조하세요.

이 가이드에서는 다음 사용 사례를 다룹니다.

- 클러스터링된 모델을 정의합니다.
- 클러스터링된 모델을 검사하고 역직렬화합니다.
- 클러스터링된 모델의 정확성을 개선합니다.
- 배포의 경우에만 압축의 이점을 확인하기 위한 단계를 취해야 합니다.


## 설정


In [ ]:
! pip install -q tensorflow-model-optimization

import tensorflow as tf
import numpy as np
import tempfile
import os
import tensorflow_model_optimization as tfmot

input_dim = 20
output_dim = 20
x_train = np.random.randn(1, input_dim).astype(np.float32)
y_train = tf.keras.utils.to_categorical(np.random.randn(1), num_classes=output_dim)

def setup_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(input_dim, input_shape=[input_dim]),
      tf.keras.layers.Flatten()
  ])
  return model

def train_model(model):
  model.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer='adam',
      metrics=['accuracy']
  )
  model.summary()
  model.fit(x_train, y_train)
  return model

def save_model_weights(model):
  _, pretrained_weights = tempfile.mkstemp('.h5')
  model.save_weights(pretrained_weights)
  return pretrained_weights

def setup_pretrained_weights():
  model= setup_model()
  model = train_model(model)
  pretrained_weights = save_model_weights(model)
  return pretrained_weights

def setup_pretrained_model():
  model = setup_model()
  pretrained_weights = setup_pretrained_weights()
  model.load_weights(pretrained_weights)
  return model

def save_model_file(model):
  _, keras_file = tempfile.mkstemp('.h5') 
  model.save(keras_file, include_optimizer=False)
  return keras_file

def get_gzipped_model_size(model):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  keras_file = save_model_file(model)

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(keras_file)
  return os.path.getsize(zipped_file)

setup_model()
pretrained_weights = setup_pretrained_weights()

## 클러스터링된 모델 정의하기


### 전체 모델 클러스터링(순차적 및 함수형)

모델 정확성 개선을 위한 **팁**:

- 이 API에 수용 가능한 정확성으로 미리 훈련된 모델을 전달해야 합니다. 클러스터링을 사용하여 처음부터 모델을 훈련하면 정확성이 떨어집니다.
- 경우에 따라, 특정 레이어를 클러스터링하면 모델 정확성에 부정적인 영향을 미칩니다. 정확성에 가장 큰 영향을 미치는 레이어 클러스터링을 건너뛰는 방법을 보려면 "일부 레이어 클러스터링"을 확인하세요.

모든 레이어를 클러스터링하려면 `tfmot.clustering.keras.cluster_weights`를 모델에 적용합니다.


In [ ]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 3,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

model = setup_model()
model.load_weights(pretrained_weights)

clustered_model = cluster_weights(model, **clustering_params)

clustered_model.summary()

### 일부 레이어 클러스터링(순차적 및 기능적 모델)


모델 정확성 개선을 위한 **팁**:

- You must pass a pre-trained model with acceptable accuracy to this API. Training models from scratch with clustering results in subpar accuracy.
- 초기 레이어와 달리 더 많은 중복 매개변수(예: `tf.keras.layers.Dense`, `tf.keras.layers.Conv2D`)를 가진 이후 레이어를 클러스터링합니다.
- 미세 조정 중 클러스터링된 레이어 이전에 초기 레이어를 고정합니다. 고정된 레이어의 수를 하이퍼 매개변수로 취급합니다. 경험적으로, 대부분의 초기 레이어를 고정하는 것이 현재의 클러스터링 API에 이상적입니다.
- 중요 레이어(예: 주의 메커니즘)의 클러스터링을 피합니다.

**추가 정보**: `tfmot.clustering.keras.cluster_weights` API 문서에서 레이어별로 클러스터링 구성을 변경하는 방법에 대한 세부 정보를 제공합니다.

In [ ]:
# Create a base model
base_model = setup_model()
base_model.load_weights(pretrained_weights)

# Helper function uses `cluster_weights` to make only 
# the Dense layers train with clustering
def apply_clustering_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return cluster_weights(layer, **clustering_params)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_clustering_to_dense` 
# to the layers of the model.
clustered_model = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_clustering_to_dense,
)

clustered_model.summary()

### 사용자 정의 Keras 레이어 클러스터링 또는 클러스트링 대상 레이어의 가중치 지정하기

`tfmot.clustering.keras.ClusterableLayer`는 두 가지 사용 사례를 제공합니다.

1. 사용자 정의 Keras 레이어를 포함하여 기본적으로 지원하지 않는 모든 레이어를 클러스터링합니다.
2. 클러스터링할 지원 레이어의 가중치를 지정합니다.

예를 들어 API는 기본적으로 `Dense` 레이어의 커널만 클러스터링합니다. 아래의 예시는 바이어스를 클러스터링하도록 이를 수정하는 방법을 보여줍니다. Keras 레이어에서 파생할 경우 `get_clusterable_weights` 함수를 재정의해야 합니다. 여기서 클러스터링할 학습 가능한 변수의 이름과 학습 가능한 변수 자체를 지정합니다. 예를 들어 빈 목록 []을 반환하면 가중치를 클러스터링할 수 없습니다.

**일반적인 실수:** 바이어스를 클러스터링하면 일반적으로 모델 정확성이 너무 많이 손상됩니다.

In [ ]:
class MyDenseLayer(tf.keras.layers.Dense, tfmot.clustering.keras.ClusterableLayer):

  def get_clusterable_weights(self):
   # Cluster kernel and bias. This is just an example, clustering
   # bias usually hurts model accuracy.
   return [('kernel', self.kernel), ('bias', self.bias)]

# Use `cluster_weights` to make the `MyDenseLayer` layer train with clustering as usual.
model_for_clustering = tf.keras.Sequential([
  tfmot.clustering.keras.cluster_weights(MyDenseLayer(20, input_shape=[input_dim]), **clustering_params),
  tf.keras.layers.Flatten()
])

model_for_clustering.summary()

`tfmot.clustering.keras.ClusterableLayer`를 사용하여 Keras 사용자 정의 레이어를 클러스터링할 수도 있습니다. 이렇게 하려면 평소와 같이 `tf.keras.Layer`를 확장하고 `__init__`, `call`, `build` 함수를 구현합니다. 단, `clusterable_layer.ClusterableLayer` 클래스도 확장하고 다음도 구현해야 합니다.

1. `get_clusterable_weights`, 여기에서 위와 같이 클러스터링할 가중치 커널을 지정합니다.
2. `get_clusterable_algorithm`, 여기에서 가중치 텐서의 클러스터링 알고리즘을 지정합니다. 이는 클러스터링을 위해 사용자 정의 레이어 가중치의 모양을 지정해야 하기 때문입니다. 반환된 클러스터링 알고리즘 클래스는 `clustering_algorithm.ClusteringAlgorithm` 클래스에서 파생되어야 하며 `get_pulling_indices` 함수를 덮어써야 합니다. 1D, 2D, 3D 등급의 가중치를 지원하는 이 함수의 예시는 [여기](https://github.com/tensorflow/model-optimization/blob/18e87d262e536c9a742aef700880e71b47a7f768/tensorflow_model_optimization/python/core/clustering/keras/clustering_algorithm.py#L62)에서 확인할 수 있습니다.

이 사용 사례의 예시는 [여기](https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/python/core/clustering/keras/mnist_clusterable_layer_test.py)에서 확인할 수 있습니다.

## 클러스터링된 모델 검사 및 역직렬화하기

**사용 사례:** 이 코드는 HDF5 모델 형식에만 필요합니다(HDF5 가중치 또는 기타 형식은 해당되지 않음).

In [ ]:
# Define the model.
base_model = setup_model()
base_model.load_weights(pretrained_weights)
clustered_model = cluster_weights(base_model, **clustering_params)

# Save or checkpoint the model.
_, keras_model_file = tempfile.mkstemp('.h5')
clustered_model.save(keras_model_file, include_optimizer=True)

# `cluster_scope` is needed for deserializing HDF5 models.
with tfmot.clustering.keras.cluster_scope():
  loaded_model = tf.keras.models.load_model(keras_model_file)

loaded_model.summary()

## 클러스터링된 모델의 정확성 개선하기

특정 사용 사례에 대해 고려할 수 있는 팁을 소개합니다.

- 센트로이드 초기화(Centroid initialization)는 최종 최적화된 모델의 정확성에서 중요한 역할을 합니다. 일반적으로 kmeans++ 초기화는 선형, 밀도 및 임의 초기화보다 성능이 우수합니다.  kmeans++를 사용하지 않을 경우 선형 초기화는 큰 가중치를 놓치는 경향이 없기 때문에 밀도 및 임의 초기화보다 성능이 더 우수합니다. 그러나, 밀도 초기화는 바이모달 분포가 있는 가중치에 클러스터를 거의 사용하지 않는 경우 더 나은 정확성을 제공하는 것으로 관찰되었습니다.

- 클러스터링된 모델을 미세 조정할 때 훈련에 사용되는 학습률보다 낮은 학습률을 설정합니다.

- 모델 정확성을 개선하기 위한 일반적인 아이디어를 얻으려면 "클러스터링된 모델 정의하기"에서 사용 사례에 대한 팁을 살펴보세요.

## 배포

### 크기 압축으로 모델 내보내기

**일반적인 실수**: 클러스터링의 압축 이점을 확인하려면 <code>strip_clustering</code>과 표준 압축 알고리즘(예: gzip 이용) 적용이 모두 필요합니다.

In [ ]:
model = setup_model()
clustered_model = cluster_weights(model, **clustering_params)

clustered_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

clustered_model.fit(
    x_train,
    y_train
)

final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

print("final model")
final_model.summary()

print("\n")
print("Size of gzipped clustered model without stripping: %.2f bytes" 
      % (get_gzipped_model_size(clustered_model)))
print("Size of gzipped clustered model with stripping: %.2f bytes" 
      % (get_gzipped_model_size(final_model)))